In [ ]:
from elasticsearch import Elasticsearch

In [ ]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic","3TgCUfRLY3o7BXKYpWFs"),
    ca_certs="/home/isurika/downloads/elasticsearch-8.12.0/config/certs/http_ca.crt"
)
es.ping()

### Prepare the data

In [ ]:
import pandas as pd

df = pd.read_csv("/mnt/d/University/repos/aws-recommendation-system/data_collection/ec2_data.csv").loc[:1500]
df.head()

#### Check NA values


In [ ]:
df.isna().value_counts()

In [ ]:
df.fillna("None", inplace=True)

### Convert the relevant field to Vector using BERT model

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
df["DescriptionVector"] = df["response"].apply(lambda x: model.encode(x))

In [ ]:
df.head()

In [ ]:
es.ping()

### Create new index in ElasticSearch

In [ ]:
from indexMapping import indexMapping

es.indices.create(index="all_patterns_1500", mappings=indexMapping)

### Ingest the data into index

In [ ]:
record_list = df.to_dict("records")

In [ ]:
for record in record_list:
    try:
        es.index(index="all_patterns_1500", document=record, id=record["id"])
    except Exception as e:
        print(e)

In [ ]:
es.count(index="all_patterns_1500")

### Search the data

In [ ]:
input_keyword = " Billing of Amazon EC2 systems begin and end?"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field": "DescriptionVector",
    "query_vector": vector_of_input_keyword,
    "k": 1,  # Set k to 1 to get only the top result
    "num_candidates": 1500,
}

res = es.knn_search(index="all_patterns_1500", knn=query, source=["pattern", "response"])
hits = res["hits"]["hits"]


if hits:
    best_match = hits[0]
    print("Best Matching Result:")
    print("Pattern:", best_match["_source"]["pattern"])
    print("Response:", best_match["_source"]["response"])
else:
    print("No matching results found.")
